In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import random
matplotlib.rcParams['figure.figsize'] = (5,5)
import pandas as pd
import networkx as nx

labels = pd.read_csv("/home/nicholas/GitHub/yabacus/seq.tab", sep="\t")
distances = pd.read_csv("/home/nicholas/GitHub/yabacus/simulatedGenomeResults_5/2rDNA/mapped.tab", sep="\t", 
    names=["query", "subject", "perc", "length", "qs", "ss", "qe", 
            "se","a","b", "eval", "bitscore"])

START = False
cluster_dict = {}
with open("/home/nicholas/GitHub/yabacus/out.seq.mci.I14", "r") as inf:
    cluster = 0
    for idx, line in enumerate(inf):
        if idx < 7:
            continue
        if line.startswith(")"):
            break
        if line.startswith(" "):
            cluster_dict[cluster].extend([int(x) for x in line.split() if x != "$"]) 
        else:
            cluster = int(line.split()[0])
            cluster_dict[cluster] = [int(x) for x in line.split()[1:] if x != "$"]
#print(cluster_dict)


def talldf_from_dict(cluster_dict):
    """retuns a dict
    """
    df = pd.DataFrame.from_dict(cluster_dict, orient="index").head().melt(
        var_name='cluster', value_name='id')
    return df
cluster_df = talldf_from_dict(cluster_dict=cluster_dict)
print(distances.shape)
print(cluster_df.shape)

genome_length = 20000
depth = int(len(distances) / genome_length) 
#distances = distances.iloc[random.sample(list(distances.index), len(distances))]
print(distances.shape)
print("Depth: %d" %depth)
print(distances.head())

(1001260, 12)
(1200, 2)
(1001260, 12)
Depth: 50
                      query                   subject   perc  length  qs  ss  \
0  concatenated_genome-9000  concatenated_genome-8862  100.0     100   0   0   
1  concatenated_genome-9000  concatenated_genome-9000  100.0     100   0   0   
2  concatenated_genome-9000  concatenated_genome-3775  100.0      99   0   0   
3  concatenated_genome-9000  concatenated_genome-5445  100.0      98   0   0   
4  concatenated_genome-9000  concatenated_genome-6011  100.0      98   0   0   

   qe   se    a    b          eval  bitscore  
0   1  100    1  100  1.000000e-50     192.0  
1   1  100    1  100  1.000000e-50     192.0  
2   2  100  100    2  4.000000e-50     191.0  
3   1   98   98    1  2.000000e-49     189.0  
4   3  100    1   98  2.000000e-49     189.0  


In [2]:
gff = "/home/nicholas/GitHub/yabacus/simulatedGenomeResults_5/toyGenome/coli_genome/scan/scannedScaffolds.gff"
gff_lines = [["seq", "rdna_start", "rdna_end", "product", "color"]] # [seq, start, product]
def choose_color(rRNA):
    if "16" in rRNA:
        return "red"
    elif "23" in rRNA:
        return "blue"
    elif "5" in rRNA:
        return "green"
    else:
        return "black"
with open(gff, "r") as inf:
    for line in inf:
        if not line.startswith("#"): 
            ff = line.split()
            product = ff[8].split("product=")[1]
            gff_lines.append([ff[0], int(ff[3]), int(ff[4]), product, choose_color(product)])
gffdf = pd.DataFrame(gff_lines[1:], columns=gff_lines[0])
print(gffdf.head())

                     seq  rdna_start  rdna_end product  color
0  concatenated_genome_0        2001      3538     16S    red
1  concatenated_genome_0        3988      6887     23S   blue
2  concatenated_genome_0        6985      7095      5S  green
3  concatenated_genome_0       11096     12633     16S    red
4  concatenated_genome_0       12991     15890     23S   blue


In [3]:
sam = "/home/nicholas/GitHub/yabacus/simulatedGenomeResults_5/2rDNA/mapped_to_short_ref.sam"
sam_lines = [["query", "start"]]  # [read_id, start]
with open(sam, "r") as insam:
    for line in insam:
        if not line.startswith("@"):
#            print(fields)
            fields = line.split()
            sam_lines.append([fields[0], int(fields[3])])
samdf = pd.DataFrame(sam_lines[1:], columns=sam_lines[0])
print(samdf.head())

rDNA_mask, color_mask = [], []
for i, row in samdf.iterrows():
    rDNA, this_color = False, "black"
    for j, grow in gffdf.iterrows():
        if grow['rdna_start'] <= row['start'] <= grow['rdna_end']:
            rDNA = True
            this_color = grow['color']
    color_mask.append(this_color)
    rDNA_mask.append(rDNA)
samdf['rdna'] = rDNA_mask
samdf['color'] = color_mask


print(samdf.head())

                      query  start
0  concatenated_genome-9000   5761
1  concatenated_genome-8999   5158
2  concatenated_genome-8998   8076
3  concatenated_genome-8997    254
4  concatenated_genome-8996   8343
                      query  start   rdna  color
0  concatenated_genome-9000   5761   True   blue
1  concatenated_genome-8999   5158   True   blue
2  concatenated_genome-8998   8076  False  black
3  concatenated_genome-8997    254  False  black
4  concatenated_genome-8996   8343  False  black


In [4]:
df_tmp = pd.merge(distances, samdf, how="left", on="query")
df_tmp['query_color'] = df_tmp['color']
df = pd.merge(df_tmp[["query", "subject", "perc", "length", "qs", "ss", "qe", "se", "a", "b", "eval", "bitscore", "query_color"]],
              samdf, how="left", left_on="subject", right_on="query")
df['subject_color'] = df['color']
df['query'] = df['query_x']

print(df.head())
print(df.shape)


                    query_x                   subject   perc  length  qs  ss  \
0  concatenated_genome-9000  concatenated_genome-8862  100.0     100   0   0   
1  concatenated_genome-9000  concatenated_genome-9000  100.0     100   0   0   
2  concatenated_genome-9000  concatenated_genome-3775  100.0      99   0   0   
3  concatenated_genome-9000  concatenated_genome-5445  100.0      98   0   0   
4  concatenated_genome-9000  concatenated_genome-6011  100.0      98   0   0   

   qe   se    a    b          eval  bitscore query_color  \
0   1  100    1  100  1.000000e-50     192.0        blue   
1   1  100    1  100  1.000000e-50     192.0        blue   
2   2  100  100    2  4.000000e-50     191.0        blue   
3   1   98   98    1  2.000000e-49     189.0        blue   
4   3  100    1   98  2.000000e-49     189.0        blue   

                    query_y  start  rdna color subject_color  \
0  concatenated_genome-8862   5761  True  blue          blue   
1  concatenated_genome-9000   

In [8]:
def plot_graph_at_depth(df, depth, x):
    color_map = []
    print("processing %d" %x)
    print("realtive amount: %f" % x * depth)
    mask = df.groupby('query').query.transform('count') >  x * depth
    this_distances = df[mask]
    print(this_distances.shape)
    if len(this_distances) > 100000:
        indexes = random.sample(list(this_distances.index), 100000)
        this_distances = this_distances.loc[indexes]
    print(this_distances.shape)
    G = nx.Graph()
    for idx, row in this_distances.iterrows():
        if idx > 4000000:
            break
        if row["query"] not in G.nodes:
            color_map.append(row['query_color'])
            G.add_node(row["query"])
        if row["subject"] not in G.nodes:
            color_map.append(row['subject_color'])
            G.add_node(row["subject"])
        G.add_edge(row["subject"], row["query"], length=row['bitscore'])
    nx.draw(G, font_weight='bold', node_size=1, edge_width=2, edge_color="grey", node_color = color_map, with_labels = False)
    plt.savefig(fname="network_retained_depth_%.2f_%.2fx.png" % (x, depth * x),
                dpi=200,
                orientation='portrait', papertype=None, format="png",
                transparent=False, bbox_inches=None, pad_inches=0.1,
                frameon=None)
    plt.close()
#plot_graph_at_depth(df, depth=depth, x=8)

In [9]:
# need to filter hits with less than 10 neighbors
rels = [x * .01 for x in range(0, 400, 20)]
print(rels)
import multiprocessing
pool = multiprocessing.Pool(processes=4)
results = [
    pool.apply_async(plot_graph_at_depth,
                     (df,),
                     {#"df": distances,
                      "depth": depth,
                      "x": this_depth})
    for this_depth in rels]
pool.close()
pool.join()
#subassembly_return_sum = sum([r.get() for r in results])




[0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4000000000000001, 1.6, 1.8, 2.0, 2.2, 2.4, 2.6, 2.8000000000000003, 3.0, 3.2, 3.4, 3.6, 3.8000000000000003]
processing 0
realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount: 0.000000realtive amount:

processing 1
(976715, 19)
(100000, 19)
processing 1
realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.400000realtive amount: 1.4000

(632104, 19)
(100000, 19)
processing 2
realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive amount: 2.800000realtive am

In [ ]:
results[0].__dict__
